In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import os
from tqdm import tqdm_notebook as tqdm
#from codecs import open
from lxml import etree

from nlppln.utils import out_file_name, create_dirs

In [ ]:
#fname = '_aio001jver01_01.txt'
#fname = 'rade001gera01_01.txt'
#fname = '_zev001198901_01.txt'
#fname = '_tir001196201_01.txt'
fname = 'looy001wond03_01.txt'

with open('/home/jvdzwaan/data/dbnl_ocr/raw/ocr-without-title-page/{}'.format(fname)) as f:
    ls = f.readlines()

with open('/home/jvdzwaan/data/dbnl_ocr/raw/notes/{}'.format(fname)) as f:
    notes = f.readlines()

print(len(ls))
print(len(notes))

In [ ]:
# remove empty lines from lines
lines = []

for line in ls:
    if line.strip() != '':
        lines.append(line)
print(len(lines))

In [ ]:
import edlib

class Note():
    def __init__(self, text, index, source_text_length):
        self.index = index
        self.text = text
        self.source_text_length = source_text_length
        self.lines_to_check = []
        self.lines = []
        
        self.previous = None
        self.next = None
        
        self.done = False
        
    def __str__(self):
        return '(Note {}) {}'.format(self.index, self.text[:50].strip())
    
    #def start(self):
    #    if self.previous is None:
    #        return 0
    #    return self.previous.end()
    
    #def end(self):
    #    if len(self.lines) > 0:
    #        return self.lines[-1]
    #    elif self.previous is not None:
    #        return self.previous.end()
    #    return 0
    
    def get_search_start(self):
        if self.previous is None:
            return 0
        elif self.previous.lines != []:
            return self.previous.lines[-1]
        else:
            return self.previous.get_search_start()
        
    def get_search_end(self):
        if self.next is None:
            return self.source_text_length
        elif self.next.lines != []:
            return self.next.lines[0]
        else:
            return self.next.get_search_end()
            
    def get_search_indices(self):
        return range((self.get_search_start()+ 1), self.get_search_end())
            
    def to_fragment(self, lines):
        return ''.join([lines[i] for i in self.lines])
    
    def ed(self, lines):
        fragment = self.to_fragment(lines)
        return edlib.align(self.text, fragment)['editDistance']
    
    def complete(self, lines, threshold=0.2):
        ed = self.ed(lines)
        score = ed/len(self.text)
        if score < threshold:
            self.done = True
        return self.done


In [ ]:
import edlib

def to_fragment(lines, indices):
    return ''.join([lines[i] for i in indices])

def complete(note, fragment, treshold=0.2):
    ed = edlib.align(note, fragment)['editDistance']
    print(ed)
    print(len(note))
    score = ed/len(note)
    print(score)
    if score < treshold:
        return True
    return False


In [ ]:
from collections import Counter

def get_repeated(notes):
    c = Counter()
    num_lines = 0

    for note in notes:
        c[note] += 1

    repeated = []
    ns = []
    for k, v in c.most_common():
        if v > 1:
            num_lines += v
            #print(repr(k), v)
            repeated.append(k)
        else:
            ns.append(k)
    #print('total number of lines:', num_lines)
    
    return repeated

repeated = get_repeated(notes)

In [ ]:
%%time
# match entire notes: quick way to distribute the notes over the text
from py_stringmatching.similarity_measure.partial_ratio import PartialRatio
pr = PartialRatio()

max_score = 0

prev_note = None
ns = []
text = ''.join(lines)
for i, n in enumerate(notes):
    note = Note(n, i, len(lines))
    score = pr.get_raw_score(note.text, text)
    if score > max_score:
        max_score = score
    #print(i, 'score', score)
    if score > 90:
        note.found = True
    else:
        note.found = False
    if n in repeated:
        note.repeated = True
    else:
        note.repeated = False
        
    note.previous = prev_note
    prev_note = note
    
    ns.append(note)

print('max score:', max_score)

ns.reverse()
for i, n in enumerate(ns):
    if i > 0:
        n.next = ns[i-1]
ns.reverse()

| File | Max score |
| -- | -- |
| rade001gera01_01.txt | 84 |
| \_zev001198901_01.txt | 78 |
| \_tir001196201_01.txt | 59 |